### Proyecto 1 - preprocesamiento

###### Maria Isabel Ortiz Naranjo
###### Luis Alejandro Urbina
###### Catedratico: Jorge Yass
###### Catedra: Security Data Science

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas_profiling as pp

from pandas_profiling import ProfileReport
from imblearn.over_sampling import RandomOverSampler
from collections import Counter


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report, accuracy_score

from sklearn.preprocessing import StandardScaler

import sklearn
from sklearn import metrics, model_selection, tree

reports: bool = False

In [ ]:
df_final = pd.read_csv("/content/gdrive/MyDrive/DatasetsProyecto/dataset-part1.csv")

#df_final1 = pd.read_csv("/content/gdrive/MyDrive/DatasetsProyecto/dataset-part2.csv")


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3570666 entries, 0 to 3570665
Data columns (total 50 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   BIFLOW_DIRECTION            int64 
 1   DIRECTION                   int64 
 2   DST_TO_SRC_SECOND_BYTES     object
 3   FIREWALL_EVENT              int64 
 4   FIRST_SWITCHED              int64 
 5   FLOW_ACTIVE_TIMEOUT         int64 
 6   FLOW_DURATION_MICROSECONDS  int64 
 7   FLOW_DURATION_MILLISECONDS  int64 
 8   FLOW_END_MILLISECONDS       int64 
 9   FLOW_END_SEC                int64 
 10  FLOW_ID                     int64 
 11  FLOW_INACTIVE_TIMEOUT       int64 
 12  FLOW_START_MILLISECONDS     int64 
 13  FLOW_START_SEC              int64 
 14  FRAME_LENGTH                int64 
 15  IN_BYTES                    int64 
 16  IN_PKTS                     int64 
 17  IPV4_DST_ADDR               object
 18  IPV4_SRC_ADDR               object
 19  L4_DST_PORT                 int64 
 20  L4

In [ ]:
df_final.head()

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_ID,FLOW_INACTIVE_TIMEOUT,FLOW_START_MILLISECONDS,FLOW_START_SEC,FRAME_LENGTH,IN_BYTES,IN_PKTS,IPV4_DST_ADDR,IPV4_SRC_ADDR,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,MAX_IP_PKT_LEN,MIN_IP_PKT_LEN,OOORDER_IN_PKTS,OOORDER_OUT_PKTS,OUT_BYTES,OUT_PKTS,PROTOCOL,PROTOCOL_MAP,RETRANSMITTED_IN_BYTES,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,RETRANSMITTED_OUT_PKTS,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,40,0,1616660040,120,339,0,1616660040010,1616660040,2293398,30,1616660040010,1616660040,0,44,1,10.114.225.212,162.142.125.173,60541,22351,1616660040,0,0,0,0,40,1,6,tcp,0,0,0,0,44,22,1024,0,1024,0,1460,0,0,0,0,0,Unknown,1,2293398,Normal flow
1,1,0,",",0,1616660040,120,0,0,1616660040068,1616660040,2293400,30,1616660040068,1616660040,0,68,1,10.114.225.215,45.79.106.170,0,0,1616660040,0,0,0,0,0,0,1,icmp,0,0,0,0,68,0,0,0,0,0,0,0,0,0,40,0,ICMP,1,2293400,Normal flow
2,1,0,104,0,1616660040,120,44725,44,1616660040114,1616660040,2293404,30,1616660040070,1616660040,0,189,2,10.114.241.165,10.114.224.65,24650,443,1616660040,0,0,0,0,104,2,6,tcp,0,0,0,0,189,25,122,1634,122,1634,0,0,0,0,0,0,TLS,1,2293404,Normal flow
3,1,0,",",0,1616660040,120,0,0,1616660040122,1616660040,2293407,30,1616660040122,1616660040,0,52,1,10.114.226.23,202.179.91.28,8728,61075,1616660040,0,0,0,0,0,0,6,tcp,0,0,0,0,52,2,8192,0,8192,0,1440,0,8,0,40,0,Unknown,1,2293407,Normal flow
4,1,0,40,0,1616660040,120,1114,1,1616660040184,1616660040,2293409,30,1616660040183,1616660040,0,189,2,10.114.224.65,10.114.241.165,443,39762,1616660040,0,0,0,0,40,1,6,tcp,0,0,0,0,189,28,502,0,502,0,0,0,0,0,0,0,TLS,1,2293409,Normal flow


#**Preprocesamiento**
#####remover variables innecesarias

In [ ]:
vars = ['MIN_IP_PKT_LEN','SAMPLING_INTERVAL','DIRECTION','OOORDER_IN_PKTS','OOORDER_OUT_PKTS','BIFLOW_DIRECTION','FIREWALL_EVENT','FLOW_ACTIVE_TIMEOUT','FLOW_INACTIVE_TIMEOUT', 'FRAME_LENGTH', 'MAX_IP_PKT_LEN',
]

In [ ]:
df_final = df_final.drop(vars, axis=1)
len(df_final.columns.values)

39

In [ ]:
df_final = df_final.drop(["PROTOCOL"], axis = 1)

In [ ]:
unvars = ['FLOW_ID', 'TOTAL_FLOWS_EXP']
df_final = df_final.drop(unvars, axis=1)
len(df_final.columns.values)

36

In [ ]:
df_final = df_final.drop(["IPV4_DST_ADDR", "IPV4_SRC_ADDR"], axis = 1)

In [ ]:
df_final.head()

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,PROTOCOL_MAP,RETRANSMITTED_IN_BYTES,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,RETRANSMITTED_OUT_PKTS,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,LABEL
0,40,1616660040,339,0,1616660040010,1616660040,1616660040010,1616660040,44,1,60541,22351,1616660040,40,1,tcp,0,0,0,0,44,22,1024,0,1024,0,1460,0,0,0,0,0,Unknown,Normal flow
1,",",1616660040,0,0,1616660040068,1616660040,1616660040068,1616660040,68,1,0,0,1616660040,0,0,icmp,0,0,0,0,68,0,0,0,0,0,0,0,0,0,40,0,ICMP,Normal flow
2,104,1616660040,44725,44,1616660040114,1616660040,1616660040070,1616660040,189,2,24650,443,1616660040,104,2,tcp,0,0,0,0,189,25,122,1634,122,1634,0,0,0,0,0,0,TLS,Normal flow
3,",",1616660040,0,0,1616660040122,1616660040,1616660040122,1616660040,52,1,8728,61075,1616660040,0,0,tcp,0,0,0,0,52,2,8192,0,8192,0,1440,0,8,0,40,0,Unknown,Normal flow
4,40,1616660040,1114,1,1616660040184,1616660040,1616660040183,1616660040,189,2,443,39762,1616660040,40,1,tcp,0,0,0,0,189,28,502,0,502,0,0,0,0,0,0,0,TLS,Normal flow


In [ ]:
df_final["LABEL"].unique()


array(['Normal flow', 'SYN Scan - aggressive'], dtype=object)

In [ ]:
df_final.describe(include=['object'])

,DST_TO_SRC_SECOND_BYTES,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,L7_PROTO_NAME,LABEL
count,3570666,3570666,3570666,3570666,3570666
unique,122390,5,133551,360,2
top,40,tcp,44,Unknown,SYN Scan - aggressive
freq,1953162,2999986,2498365,2717179,2496814


In [ ]:
objectss = df_final.select_dtypes(include=['object']).columns
df_final[objectss].head(50)

,DST_TO_SRC_SECOND_BYTES,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,L7_PROTO_NAME,LABEL
0,40,tcp,44,Unknown,Normal flow
1,",",icmp,68,ICMP,Normal flow
2,104,tcp,189,TLS,Normal flow
3,",",tcp,52,Unknown,Normal flow
4,40,tcp,189,TLS,Normal flow
5,40,tcp,189,TLS,Normal flow
6,5043,tcp,3699,TLS.Google,Normal flow
7,1009,tcp,4861,TLS.Teams,Normal flow
8,",",tcp,52,SMBv23,Normal flow
9,120,tcp,5707,TLS.Skype,Normal flow


In [ ]:
df_final.head()

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,PROTOCOL_MAP,RETRANSMITTED_IN_BYTES,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,RETRANSMITTED_OUT_PKTS,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,LABEL
0,40,1616660040,339,0,1616660040010,1616660040,1616660040010,1616660040,44,1,60541,22351,1616660040,40,1,tcp,0,0,0,0,44,22,1024,0,1024,0,1460,0,0,0,0,0,Unknown,Normal flow
1,",",1616660040,0,0,1616660040068,1616660040,1616660040068,1616660040,68,1,0,0,1616660040,0,0,icmp,0,0,0,0,68,0,0,0,0,0,0,0,0,0,40,0,ICMP,Normal flow
2,104,1616660040,44725,44,1616660040114,1616660040,1616660040070,1616660040,189,2,24650,443,1616660040,104,2,tcp,0,0,0,0,189,25,122,1634,122,1634,0,0,0,0,0,0,TLS,Normal flow
3,",",1616660040,0,0,1616660040122,1616660040,1616660040122,1616660040,52,1,8728,61075,1616660040,0,0,tcp,0,0,0,0,52,2,8192,0,8192,0,1440,0,8,0,40,0,Unknown,Normal flow
4,40,1616660040,1114,1,1616660040184,1616660040,1616660040183,1616660040,189,2,443,39762,1616660040,40,1,tcp,0,0,0,0,189,28,502,0,502,0,0,0,0,0,0,0,TLS,Normal flow


Borrar columnas con 0 

In [ ]:
df_final = df_final.drop(columns=["RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS"], axis=1)
df_final = df_final.drop(columns=["RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS"], axis = 1)
df_final = df_final.drop(columns=["TCP_WIN_MSS_OUT", "TCP_WIN_SCALE_IN", "TCP_WIN_SCALE_IN", "TCP_WIN_SCALE_OUT", "SRC_TOS", "DST_TOS"], axis=1)

In [ ]:
# show the dataset
df_final.head(
)

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,L7_PROTO_NAME,LABEL
0,40,1616660040,339,0,1616660040010,1616660040,1616660040010,1616660040,44,1,60541,22351,1616660040,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown,Normal flow
1,",",1616660040,0,0,1616660040068,1616660040,1616660040068,1616660040,68,1,0,0,1616660040,0,0,icmp,68,0,0,0,0,0,0,ICMP,Normal flow
2,104,1616660040,44725,44,1616660040114,1616660040,1616660040070,1616660040,189,2,24650,443,1616660040,104,2,tcp,189,25,122,1634,122,1634,0,TLS,Normal flow
3,",",1616660040,0,0,1616660040122,1616660040,1616660040122,1616660040,52,1,8728,61075,1616660040,0,0,tcp,52,2,8192,0,8192,0,1440,Unknown,Normal flow
4,40,1616660040,1114,1,1616660040184,1616660040,1616660040183,1616660040,189,2,443,39762,1616660040,40,1,tcp,189,28,502,0,502,0,0,TLS,Normal flow


*Remplazar 0 por 1*

In [ ]:
df_final["DST_TO_SRC_SECOND_BYTES"].replace({0: 1}, inplace=True)
df_final["DST_TO_SRC_SECOND_BYTES"].head()

0     40
1      ,
2    104
3      ,
4     40
Name: DST_TO_SRC_SECOND_BYTES, dtype: object

In [ ]:
df_final.head()

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,L7_PROTO_NAME,LABEL
0,40,1616660040,339,0,1616660040010,1616660040,1616660040010,1616660040,44,1,60541,22351,1616660040,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown,Normal flow
1,",",1616660040,0,0,1616660040068,1616660040,1616660040068,1616660040,68,1,0,0,1616660040,0,0,icmp,68,0,0,0,0,0,0,ICMP,Normal flow
2,104,1616660040,44725,44,1616660040114,1616660040,1616660040070,1616660040,189,2,24650,443,1616660040,104,2,tcp,189,25,122,1634,122,1634,0,TLS,Normal flow
3,",",1616660040,0,0,1616660040122,1616660040,1616660040122,1616660040,52,1,8728,61075,1616660040,0,0,tcp,52,2,8192,0,8192,0,1440,Unknown,Normal flow
4,40,1616660040,1114,1,1616660040184,1616660040,1616660040183,1616660040,189,2,443,39762,1616660040,40,1,tcp,189,28,502,0,502,0,0,TLS,Normal flow


In [ ]:
if not os.path.isfile("report.html") and reports:
    profile = ProfileReport(df_final, title="report", minimal=True)
    profile.to_file("report.html")

In [ ]:
df_final["LABEL"].unique()
labels_dic = {
    "Normal flow": 0,
    "SYN Scan - aggressive": 1,
    "Denial of Service R-U-Dead-Yet": 2,
    "Denial of Service Slowloris": 3
}
df_final["LABEL"] = df_final["LABEL"].apply(lambda x: labels_dic[x])
df_final["LABEL"].unique()

array([0, 1])

Colocar correctamente el nombre de las columnas

In [ ]:
df_final.columns = [column.replace("-", "_") for column in df_final.columns]
df_final.columns

Index(['DST_TO_SRC_SECOND_BYTES', 'FIRST_SWITCHED',
       'FLOW_DURATION_MICROSECONDS', 'FLOW_DURATION_MILLISECONDS',
       'FLOW_END_MILLISECONDS', 'FLOW_END_SEC', 'FLOW_START_MILLISECONDS',
       'FLOW_START_SEC', 'IN_BYTES', 'IN_PKTS', 'L4_DST_PORT', 'L4_SRC_PORT',
       'LAST_SWITCHED', 'OUT_BYTES', 'OUT_PKTS', 'PROTOCOL_MAP',
       'SRC_TO_DST_SECOND_BYTES', 'TCP_FLAGS', 'TCP_WIN_MAX_IN',
       'TCP_WIN_MAX_OUT', 'TCP_WIN_MIN_IN', 'TCP_WIN_MIN_OUT',
       'TCP_WIN_MSS_IN', 'L7_PROTO_NAME', 'LABEL'],
      dtype='object')

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3570666 entries, 0 to 3570665
Data columns (total 25 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   DST_TO_SRC_SECOND_BYTES     object
 1   FIRST_SWITCHED              int64 
 2   FLOW_DURATION_MICROSECONDS  int64 
 3   FLOW_DURATION_MILLISECONDS  int64 
 4   FLOW_END_MILLISECONDS       int64 
 5   FLOW_END_SEC                int64 
 6   FLOW_START_MILLISECONDS     int64 
 7   FLOW_START_SEC              int64 
 8   IN_BYTES                    int64 
 9   IN_PKTS                     int64 
 10  L4_DST_PORT                 int64 
 11  L4_SRC_PORT                 int64 
 12  LAST_SWITCHED               int64 
 13  OUT_BYTES                   int64 
 14  OUT_PKTS                    int64 
 15  PROTOCOL_MAP                object
 16  SRC_TO_DST_SECOND_BYTES     object
 17  TCP_FLAGS                   int64 
 18  TCP_WIN_MAX_IN              int64 
 19  TCP_WIN_MAX_OUT             int64 
 20  TC

In [ ]:
df_final.head()

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,L7_PROTO_NAME,LABEL
0,40,1616660040,339,0,1616660040010,1616660040,1616660040010,1616660040,44,1,60541,22351,1616660040,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown,0
1,",",1616660040,0,0,1616660040068,1616660040,1616660040068,1616660040,68,1,0,0,1616660040,0,0,icmp,68,0,0,0,0,0,0,ICMP,0
2,104,1616660040,44725,44,1616660040114,1616660040,1616660040070,1616660040,189,2,24650,443,1616660040,104,2,tcp,189,25,122,1634,122,1634,0,TLS,0
3,",",1616660040,0,0,1616660040122,1616660040,1616660040122,1616660040,52,1,8728,61075,1616660040,0,0,tcp,52,2,8192,0,8192,0,1440,Unknown,0
4,40,1616660040,1114,1,1616660040184,1616660040,1616660040183,1616660040,189,2,443,39762,1616660040,40,1,tcp,189,28,502,0,502,0,0,TLS,0


In [ ]:
ros = RandomOverSampler(sampling_strategy='minority')
flow = df_final.loc[df_final["LABEL"] == 0]
at = df_final.loc[df_final["LABEL"] == 1]
dead = df_final.loc[df_final["LABEL"] == 2]
att = df_final.loc[df_final["LABEL"] == 3]
print(len(flow))
print(len(at))
print(len(dead))
print(len(att))


1073852
2496814
0
0


In [ ]:
df_or = df_final.groupby('LABEL', group_keys=False).apply(lambda x: x.sample(frac=0.01))

In [ ]:
df_final = pd.concat([flow, at, dead, att])
df_final.to_csv("df_ffinal.csv")

Modelo 1



In [ ]:
# Codigo inspirado de: https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/
y = df_final['LABEL']
X = df_final.drop(['LABEL'], axis=1)

print('Final features:', X.columns)
X.head()


Final features: Index(['DST_TO_SRC_SECOND_BYTES', 'FIRST_SWITCHED',
       'FLOW_DURATION_MICROSECONDS', 'FLOW_DURATION_MILLISECONDS',
       'FLOW_END_MILLISECONDS', 'FLOW_END_SEC', 'FLOW_START_MILLISECONDS',
       'FLOW_START_SEC', 'IN_BYTES', 'IN_PKTS', 'L4_DST_PORT', 'L4_SRC_PORT',
       'LAST_SWITCHED', 'OUT_BYTES', 'OUT_PKTS', 'PROTOCOL_MAP',
       'SRC_TO_DST_SECOND_BYTES', 'TCP_FLAGS', 'TCP_WIN_MAX_IN',
       'TCP_WIN_MAX_OUT', 'TCP_WIN_MIN_IN', 'TCP_WIN_MIN_OUT',
       'TCP_WIN_MSS_IN', 'L7_PROTO_NAME'],
      dtype='object')


,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,L7_PROTO_NAME
0,40,1616660040,339,0,1616660040010,1616660040,1616660040010,1616660040,44,1,60541,22351,1616660040,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown
1,",",1616660040,0,0,1616660040068,1616660040,1616660040068,1616660040,68,1,0,0,1616660040,0,0,icmp,68,0,0,0,0,0,0,ICMP
2,104,1616660040,44725,44,1616660040114,1616660040,1616660040070,1616660040,189,2,24650,443,1616660040,104,2,tcp,189,25,122,1634,122,1634,0,TLS
3,",",1616660040,0,0,1616660040122,1616660040,1616660040122,1616660040,52,1,8728,61075,1616660040,0,0,tcp,52,2,8192,0,8192,0,1440,Unknown
4,40,1616660040,1114,1,1616660040184,1616660040,1616660040183,1616660040,189,2,443,39762,1616660040,40,1,tcp,189,28,502,0,502,0,0,TLS


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
validation_rate = 15/70
X_train_beta, X_validation, y_train_beta, y_validation = train_test_split(X_train, y_train, test_size=validation_rate, random_state=42)

In [ ]:
randomForest = RandomForestClassifier(n_estimators=1000, random_state=42)

In [ ]:
X_train_beta

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,L7_PROTO_NAME
728528,40,1616663246,696,0,1616663246144,1616663246,1616663246143,1616663246,44,1,51972,49214,1616663246,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown
741102,",",1616663292,0,0,1616663292695,1616663292,1616663292695,1616663292,44,1,23369,49215,1616663292,0,0,tcp,44,2,1024,0,1024,0,1460,Unknown
2328387,40,1616670354,488,0,1616670354685,1616670354,1616670354684,1616670354,52,1,8291,52168,1616670354,40,1,tcp,52,22,8192,0,8192,0,1452,Unknown
2041181,40,1616669432,1359,1,1616669432174,1616669432,1616669432173,1616669432,44,1,53847,49726,1616669432,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown
1777181,",",1616667978,0,0,1616667978938,1616667978,1616667978938,1616667978,84,1,0,0,1616667978,0,0,icmp,84,0,0,0,0,0,0,ICMP.Amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953581,40,1616664099,1108,1,1616664099317,1616664099,1616664099316,1616664099,44,1,41269,49214,1616664099,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown
3148852,",",1616673185,0,0,1616673185796,1616673185,1616673185796,1616673185,44,1,39979,49983,1616673185,0,0,tcp,44,2,1024,0,1024,0,1460,Unknown
3338872,",",1616673617,0,0,1616673617174,1616673617,1616673617174,1616673617,44,1,57485,49983,1616673617,0,0,tcp,44,2,1024,0,1024,0,1460,Unknown
1765116,",,221",1616667933,457408,457,1616667934344,1616667934,1616667933886,1616667933,136,2,53,23610,1616667934,221,1,udp,"136,",0,0,0,0,0,0,DNS


In [ ]:
X_train_beta.drop(["PROTOCOL_MAP", "L7_PROTO_NAME","DST_TO_SRC_SECOND_BYTES","SRC_TO_DST_SECOND_BYTES"], axis = 1)

,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN
728528,1616663246,696,0,1616663246144,1616663246,1616663246143,1616663246,44,1,51972,49214,1616663246,40,1,22,1024,0,1024,0,1460
741102,1616663292,0,0,1616663292695,1616663292,1616663292695,1616663292,44,1,23369,49215,1616663292,0,0,2,1024,0,1024,0,1460
2328387,1616670354,488,0,1616670354685,1616670354,1616670354684,1616670354,52,1,8291,52168,1616670354,40,1,22,8192,0,8192,0,1452
2041181,1616669432,1359,1,1616669432174,1616669432,1616669432173,1616669432,44,1,53847,49726,1616669432,40,1,22,1024,0,1024,0,1460
1777181,1616667978,0,0,1616667978938,1616667978,1616667978938,1616667978,84,1,0,0,1616667978,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953581,1616664099,1108,1,1616664099317,1616664099,1616664099316,1616664099,44,1,41269,49214,1616664099,40,1,22,1024,0,1024,0,1460
3148852,1616673185,0,0,1616673185796,1616673185,1616673185796,1616673185,44,1,39979,49983,1616673185,0,0,2,1024,0,1024,0,1460
3338872,1616673617,0,0,1616673617174,1616673617,1616673617174,1616673617,44,1,57485,49983,1616673617,0,0,2,1024,0,1024,0,1460
1765116,1616667933,457408,457,1616667934344,1616667934,1616667933886,1616667933,136,2,53,23610,1616667934,221,1,0,0,0,0,0,0


In [ ]:
X_train_beta.replace({",": ' '}, inplace=True)


In [ ]:
X_train_beta.head()
X_train_beta.drop(['PROTOCOL_MAP'], axis = 1)

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,L7_PROTO_NAME
728528,40,1616663246,696,0,1616663246144,1616663246,1616663246143,1616663246,44,1,51972,49214,1616663246,40,1,44,22,1024,0,1024,0,1460,Unknown
741102,,1616663292,0,0,1616663292695,1616663292,1616663292695,1616663292,44,1,23369,49215,1616663292,0,0,44,2,1024,0,1024,0,1460,Unknown
2328387,40,1616670354,488,0,1616670354685,1616670354,1616670354684,1616670354,52,1,8291,52168,1616670354,40,1,52,22,8192,0,8192,0,1452,Unknown
2041181,40,1616669432,1359,1,1616669432174,1616669432,1616669432173,1616669432,44,1,53847,49726,1616669432,40,1,44,22,1024,0,1024,0,1460,Unknown
1777181,,1616667978,0,0,1616667978938,1616667978,1616667978938,1616667978,84,1,0,0,1616667978,0,0,84,0,0,0,0,0,0,ICMP.Amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953581,40,1616664099,1108,1,1616664099317,1616664099,1616664099316,1616664099,44,1,41269,49214,1616664099,40,1,44,22,1024,0,1024,0,1460,Unknown
3148852,,1616673185,0,0,1616673185796,1616673185,1616673185796,1616673185,44,1,39979,49983,1616673185,0,0,44,2,1024,0,1024,0,1460,Unknown
3338872,,1616673617,0,0,1616673617174,1616673617,1616673617174,1616673617,44,1,57485,49983,1616673617,0,0,44,2,1024,0,1024,0,1460,Unknown
1765116,",,221",1616667933,457408,457,1616667934344,1616667934,1616667933886,1616667933,136,2,53,23610,1616667934,221,1,"136,",0,0,0,0,0,0,DNS


In [ ]:
X_train_beta.drop(['DST_TO_SRC_SECOND_BYTES'], axis = 1)

,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,L7_PROTO_NAME
728528,1616663246,696,0,1616663246144,1616663246,1616663246143,1616663246,44,1,51972,49214,1616663246,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown
741102,1616663292,0,0,1616663292695,1616663292,1616663292695,1616663292,44,1,23369,49215,1616663292,0,0,tcp,44,2,1024,0,1024,0,1460,Unknown
2328387,1616670354,488,0,1616670354685,1616670354,1616670354684,1616670354,52,1,8291,52168,1616670354,40,1,tcp,52,22,8192,0,8192,0,1452,Unknown
2041181,1616669432,1359,1,1616669432174,1616669432,1616669432173,1616669432,44,1,53847,49726,1616669432,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown
1777181,1616667978,0,0,1616667978938,1616667978,1616667978938,1616667978,84,1,0,0,1616667978,0,0,icmp,84,0,0,0,0,0,0,ICMP.Amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953581,1616664099,1108,1,1616664099317,1616664099,1616664099316,1616664099,44,1,41269,49214,1616664099,40,1,tcp,44,22,1024,0,1024,0,1460,Unknown
3148852,1616673185,0,0,1616673185796,1616673185,1616673185796,1616673185,44,1,39979,49983,1616673185,0,0,tcp,44,2,1024,0,1024,0,1460,Unknown
3338872,1616673617,0,0,1616673617174,1616673617,1616673617174,1616673617,44,1,57485,49983,1616673617,0,0,tcp,44,2,1024,0,1024,0,1460,Unknown
1765116,1616667933,457408,457,1616667934344,1616667934,1616667933886,1616667933,136,2,53,23610,1616667934,221,1,udp,"136,",0,0,0,0,0,0,DNS


In [ ]:
X_train_beta.replace({" ": ''}, inplace=True)

In [ ]:
#del(X_train_beta['L7_PROTO_NAME'])
X_train_beta.head()
#del(X_train_beta['PROTOCOL_MAP'])
#del(X_train_beta['DST_TO_SRC_SECOND_BYTES'])
#del(X_train_beta['SRC_TO_DST_SECOND_BYTES'])
X_train_beta
randomForest.fit(X_train_beta, y_train_beta)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [ ]:
y_train_beta
target_pred = clf.predict(X_train_beta)

In [ ]:
matrix = confusion_matrix(y_validation, predictions)
valrecall_score = recall_score(y_validation, predictions, average=None)
valprecision_score = precision_score(y_validation, predictions, average=None)
validation_f1_score = f1_score(y_validation, predictions, average=None)
print("Matriz confusión: ", validation_confusion_matrix)
print(classification_report(y_validation, predictions, target_names = target_names))
print("recall_score: ",recall_score)
print("precision_score: ", precision_score)
print("f1_score: ", f1_score)
print("accuracy: ", accuracy_score(y_validation, predictions))

In [ ]:
#Modelo 2 Naive Bayes
model2 = MultinomialNB()
model2.fit(X_train, Y_target_train)
pred_bow = model1.predict(x_test)
print(metrics.accuracy_score(X_target_test, pred_bow))
print('Matriz de confusion', metrics.confusion_matrix(X_target_test, pred_bow))
print(metrics.classification_report(X_target_test, pred_bow, target_names=['0', '1']))